In [2]:
import pandas as pd
df = pd.read_csv('data/daily_data.csv', parse_dates=True, index_col=0)


In [22]:
df.head()

,kWh,Voltage,Global_reactive_power,Kitchen,Laundry_Room,Water_Heater_AC,day_of_week,month,year,day_of_year,is_weekend,kWh_rolling_mean_7,kWh_rolling_std_7,kWh_lag_1,kWh_lag_7,kWh_lag_30
Datetime,,,,,,,,,,,,,,,,
2007-01-15,35.811300,241.240910,0.121431,1099.0,2239.0,15163.0,0,1,2007,15,0,38.302060,10.154146,50.183817,37.356000,20.152933
2007-01-16,28.106733,241.944757,0.128769,2063.0,644.0,6583.0,1,1,2007,16,0,37.867179,10.568206,35.811300,31.150900,56.507667
2007-01-17,47.461467,241.005250,0.139411,3500.0,5737.0,16064.0,2,1,2007,17,0,39.516912,11.100326,28.106733,35.913333,36.730433
2007-01-18,30.357700,240.893597,0.123411,0.0,336.0,13491.0,3,1,2007,18,0,38.484217,11.633308,47.461467,37.586567,27.769900
2007-01-19,24.674000,240.160944,0.118092,2103.0,330.0,7172.0,4,1,2007,19,0,38.529798,11.569169,30.357700,24.354933,37.095800


## Forecasting (Next 7 Days)